# Modelling Dynamical Systems 
## Diffusion-Limited Aggregation 

In [111]:
import random
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib import colors
import imageio
import ffmpeg
from io import BytesIO


In [15]:
def CreateRandomWalker(radius, seed_row, seed_col):

    theta = 2*np.pi*random.random()
    row=int(radius*np.cos(theta))+seed_row 
    col=int(radius*np.sin(theta))+seed_col 
    location=[row, col] 
    return location
    

In [16]:
def checkSurrounding(location, matrix_2D):

    FreeParticle = True 
    outCircle = False 
    OnEdge = False
    row = location[0]
    col = location[1]

    if (row == 0) or (col==len(matrix_2D)-1) or (row == len(matrix_2D)-1) or (col == 0):
        OnEdge = True
        
    if not OnEdge:

        if matrix_2D[row+1,col] == 1:
            FreeParticle = False
            if matrix_2D[row,col] == 2:
                outCircle = True
        elif matrix_2D[row-1,col] == 1:
            FreeParticle=False
            if matrix_2D[row,col]==2:
                outCircle=True
        elif matrix_2D[row,col+1]==1:
            FreeParticle=False
            if matrix_2D[row,col]==2:
                outCircle=True
        elif matrix_2D[row,col-1]==1:
            FreeParticle=False
            if matrix_2D[row,col]==2:
                outCircle=True

    return FreeParticle , outCircle , OnEdge
    

In [17]:
def move(location):
    
    probability = random.random()
    if probability<0.25:
        location = [location[0] - 1,location[1]]
    elif probability<0.5:
        location = [location[0] + 1,location[1]]
    elif probability<0.75:
        location = [location[0],location[1] + 1]
    else:
        location = [location[0],location[1] - 1]
        
    return location

In [131]:
def DLAsimulation_images(file_name, matrix_2D, DLAmap):
    
    if not os.path.isdir("images_and_gifs"):
        os.mkdir("images_and_gifs")
    
    fig, ax = plt.subplots()
    cax = ax.matshow(matrix_2D, interpolation='nearest', cmap=DLAmap)
    
    ax.set_title("DLA Cluster", fontsize=20)
    ax.set_xlabel("Direction, $x$", fontsize=15)
    ax.set_ylabel("Direction, $y$", fontsize=15)
    
    plt.savefig(f"images_and_gifs/{file_name}.png", dpi=200)
    
    plt.close(fig)

In [129]:
def DLAsimulation(radius, needGif):
    
    matrixLength = 2*radius + 5
    seed_row = radius + 2
    seed_col = radius + 2
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    matrix_2D[seed_row, seed_col] = 1

    distance_from_seed = np.sqrt((col - seed_col)**2 + (row - seed_row)**2)
    matrix_2D[distance_from_seed > radius] = 2
    
    randomWalkerCount = 0
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    DLAmap = colors.ListedColormap(['magenta','yellow','black'])
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        location = CreateRandomWalker(radius, seed_row, seed_col)
        randomWalkerCount += 1
        
        FreeParticle = True 
        OnEdge = False
    
        while FreeParticle and not OnEdge:
            FreeParticle, outCircle, OnEdge = checkSurrounding(location,matrix_2D)
            if OnEdge:
               randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = move(location)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if outCircle:
                    completeCluster = True
    
    if not needGif:            
        DLAsimulation_images("DLAsimulation",matrix_2D,DLAmap) 

In [136]:
DLAsimulation(100,False)


In [134]:
def DLAsimulationCircleSeed(map_radius, seed_radius, needGif):
    
    matrixLength = 2*map_radius + 5
    seed_row = map_radius + 2
    seed_col = map_radius + 2
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    distance_from_seed = np.sqrt((col - seed_col)**2 + (row - seed_row)**2)
    matrix_2D[distance_from_seed > map_radius] = 2
    matrix_2D[distance_from_seed < seed_radius] = 1
    DLAmap = colors.ListedColormap(['magenta','yellow','black'])

    randomWalkerCount = 0
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        randomWalkerCount += 1
        location = CreateRandomWalker(map_radius, seed_row, seed_col)
        
        FreeParticle = True 
        OnEdge = False
    
        while FreeParticle and not OnEdge:
            FreeParticle, outCircle, OnEdge = checkSurrounding(location,matrix_2D)
            if OnEdge:
                randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = move(location)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if outCircle:
                    completeCluster = True
    
    if not needGif:            
        DLAsimulation_images("DLAsimulation_CircleSeed",matrix_2D,DLAmap) 

In [135]:
DLAsimulationCircleSeed(100,30, False)

In [55]:
def CreateRandomWalkerCorals(depth, matrix_length):

    if ((depth<0) or (depth > matrix_length-1)):
        raise IndexError("Depth is out of range.")
    else:
        col = random.randint(0, matrix_length-1)
    return [depth,col]
    

In [54]:
def checkSurroundingCorals(location, matrix_2D):

    FreeParticle = True 
    SurfaceLayer = False 
    OnEdge = False
    row = location[0]
    col = location[1]

    if (row == len(matrix_2D)-1) or (col==len(matrix_2D)-1) or (col == 0):
        OnEdge = True
        
    if not OnEdge:

        if matrix_2D[row+1,col] == 1:
            FreeParticle = False
            if matrix_2D[row,col] == 2:
                SurfaceLayer = True
        elif matrix_2D[row-1,col] == 1:
            FreeParticle=False
        elif matrix_2D[row,col+1]==1:
            FreeParticle=False
        elif matrix_2D[row,col-1]==1:
            FreeParticle=False

    return FreeParticle , SurfaceLayer , OnEdge
    

In [53]:
def moveCorals(location):
    
    probability = random.random()
    if probability<0.33:
        location = [location[0] + 1,location[1]]
    elif probability<0.67:
        location = [location[0],location[1] + 1]
    else:
        location = [location[0],location[1] - 1]
        
    return location

In [137]:
def DLAsimulationCorals(depth, matrixLength, number_of_seeds, needGif):
    
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    if number_of_seeds < 0 or number_of_seeds > matrixLength-1:
        raise IndexError("Depth is out of range.")
    else:
        seeds = np.linspace(0,matrixLength-1,number_of_seeds,dtype=int)
       
    matrix_2D[matrixLength-1, seeds] = 1
    matrix_2D[row <= depth] = 2
    DLAmap = colors.ListedColormap(['magenta','yellow','black'])
    
    randomWalkerCount = 0
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        location = CreateRandomWalkerCorals(depth,matrixLength)
        randomWalkerCount += 1
        
        FreeParticle = True 
        OnEdge = False
    
        while FreeParticle and not OnEdge:
            FreeParticle , SurfaceLayer , OnEdge = checkSurroundingCorals(location,matrix_2D)
            if OnEdge:
               randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = moveCorals(location)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if SurfaceLayer:
                    completeCluster = True
    
    if not needGif:            
        DLAsimulation_images("DLAsimulation_Corals",matrix_2D,DLAmap) 

In [144]:
DLAsimulationCorals(30,205,30,False)

In [67]:
def checkSurroundingProbabilisticCorals(location, matrix_2D, probability):

    FreeParticle = True 
    SurfaceLayer = False 
    OnEdge = False
    row = location[0]
    col = location[1]
    Direction = "NONE"

    if (row == len(matrix_2D)-1) or (col==len(matrix_2D)-1) or (col == 0):
        OnEdge = True
        
    if not OnEdge:

        if matrix_2D[row+1,col] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    SurfaceLayer = True
            else: 
                Direction = "UP"
        elif matrix_2D[row-1,col] == 1:
            if random.random() < probability:
                FreeParticle = False
            else: 
                Direction = "DOWN"
        elif matrix_2D[row,col+1] == 1:
            if random.random() < probability:
                FreeParticle = False
            else: 
                Direction = "LEFT"
        elif matrix_2D[row,col-1] == 1:
            if random.random() < probability:
                FreeParticle = False
            else: 
                Direction = "RIGHT"

    return FreeParticle, SurfaceLayer, OnEdge, Direction

In [66]:
def moveProbabilisticCorals(location, Direction):
    match Direction:
        case "NONE":
            probability = random.random()
            if probability<0.33:
                location = [location[0] + 1,location[1]]
            elif probability<0.67:
                location = [location[0],location[1] + 1]
            else:
                location = [location[0],location[1] - 1]
        case "UP":
            location = [location[0] - 1,location[1]]
        case "DOWN":
            location = [location[0] + 1,location[1]]
        case "RIGHT":
            location = [location[0],location[1] + 1]
        case "LEFT":
            location = [location[0],location[1] - 1]
       
    return location

In [142]:
def DLAsimulationProbabilisticCorals(depth, matrixLength, number_of_seeds, probability, needGif):
    
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    if number_of_seeds < 0 or number_of_seeds > matrixLength-1:
        raise IndexError("Depth is out of range.")
    else:
        seeds = np.linspace(0,matrixLength-1,number_of_seeds,dtype=int)
       
    matrix_2D[matrixLength-1, seeds] = 1
    matrix_2D[row <= depth] = 2
    DLAmap = colors.ListedColormap(['magenta','yellow','black'])            

    
    randomWalkerCount = 0
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        randomWalkerCount += 1
        location = CreateRandomWalkerCorals(depth, matrixLength)
        
        FreeParticle = True 
        OnEdge = False
    
        while FreeParticle and not OnEdge:
            FreeParticle, SurfaceLayer, OnEdge, Direction = checkSurroundingProbabilisticCorals(location,matrix_2D,probability)
            if OnEdge:
                randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = moveProbabilisticCorals(location, Direction)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if SurfaceLayer:
                    completeCluster = True
    
    if not needGif:            
        DLAsimulation_images("DLAsimulation_Corals_probabilistic",matrix_2D,DLAmap)

In [143]:
DLAsimulationProbabilisticCorals(30,205,30,0.07,False)

In [20]:
def checkSurroundingProbabilistic(location, matrix_2D, probability):

    FreeParticle = True 
    outCircle = False 
    OnEdge = False
    row = location[0]
    col = location[1]
    Direction = "NONE"

    if (row == 0) or (col==len(matrix_2D)-1) or (row == len(matrix_2D)-1) or (col == 0):
        OnEdge = True
        
    if not OnEdge:

        if matrix_2D[row+1,col] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    outCircle = True
            else: 
                Direction = "UP"
        elif matrix_2D[row-1,col] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    outCircle = True
            else: 
                Direction = "DOWN"
        elif matrix_2D[row,col+1] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    outCircle = True
            else: 
                Direction = "LEFT"
        elif matrix_2D[row,col-1] == 1:
            if random.random() < probability:
                FreeParticle = False
                if matrix_2D[row,col] == 2:
                    outCircle = True
            else: 
                Direction = "RIGHT"

    return FreeParticle, outCircle, OnEdge, Direction

In [21]:
def moveProbabilistic(location, Direction):
    match Direction:
        case "NONE":
            probability = random.random()
            if probability<0.25:
                location = [location[0] - 1,location[1]]
            elif probability<0.5:
                location = [location[0] + 1,location[1]]
            elif probability<0.75:
                location = [location[0],location[1] + 1]
            else:
                location = [location[0],location[1] - 1]
        case "UP":
            location = [location[0] - 1,location[1]]
        case "DOWN":
            location = [location[0] + 1,location[1]]
        case "RIGHT":
            location = [location[0],location[1] + 1]
        case "LEFT":
            location = [location[0],location[1] - 1]
       
    return location

In [147]:
def DLAsimulationProbabilistic(radius, probability, needGif):
    
    matrixLength = 2*radius + 5
    seed_row = radius + 2
    seed_col = radius + 2
    matrix_2D = np.zeros((matrixLength, matrixLength))
    
    col, row = np.meshgrid(np.arange(matrixLength), np.arange(matrixLength))

    matrix_2D[seed_row, seed_col] = 1

    distance_from_seed = np.sqrt((col - seed_col)**2 + (row - seed_row)**2)
    matrix_2D[distance_from_seed > radius] = 2
    DLAmap = colors.ListedColormap(['magenta','yellow','black'])            
    
    randomWalkerCount = 0
    completeCluster = False
    maxWalkerCount = int(matrixLength**2)
    
    while not completeCluster and randomWalkerCount < maxWalkerCount:
        
        randomWalkerCount += 1
        location = CreateRandomWalker(radius, seed_row, seed_col)
        
        FreeParticle = True 
        OnEdge = False
    
        while FreeParticle and not OnEdge:
            FreeParticle, outCircle, OnEdge, Direction = checkSurroundingProbabilistic(location,matrix_2D,probability)
            if OnEdge:
                randomWalkerCount -= 1
            if FreeParticle and not OnEdge:
                location = moveProbabilistic(location, Direction)
            elif not FreeParticle:
                matrix_2D[location[0], location[1]] = 1
                if outCircle:
                    completeCluster = True
        
    if not needGif:            
        DLAsimulation_images("DLAsimulation_probabilistic",matrix_2D,DLAmap)

In [148]:
DLAsimulationProbabilistic(100, 0.07, False)